## **Minták: Több-AI Ügynökök szállodafoglaláshoz**

A mai rohanó világban egy üzleti út megtervezése nem merül ki csupán a repülőjegy és a szállodai szoba lefoglalásában. Olyan szintű koordinációt és hatékonyságot igényel, amelyet nem mindig könnyű elérni. Itt lépnek színre a Több-AI Ügynökök, forradalmasítva az utazási igényeink kezelését.

Képzelje el, hogy egy intelligens ügynökökből álló csapat áll a rendelkezésére, akik együttműködve, precízen és könnyedén kezelik utazásának minden aspektusát. Fejlett AI technológiánkkal olyan specializált ügynököket hoztunk létre, amelyek a foglalási szolgáltatásokért és az útiterv összeállításáért felelnek, biztosítva ezzel a zökkenőmentes és stresszmentes utazási élményt.

Ez egy alapvető forgatókönyv. Üzleti út tervezésekor konzultálnunk kell egy üzleti utazási ügynökkel, hogy információt szerezzünk a repülőjegyekről, szállodákról stb. Az AI Ügynökök segítségével létrehozhatunk foglalási szolgáltatásokért felelős ügynököket, valamint útiterv összeállításáért felelős ügynököket, akik együttműködve növelik az intelligencia szintjét.


# Az Azure AI Agent Service inicializálása és konfigurációs információk lekérése a **.env** fájlból

### **.env**

Hozz létre egy .env fájlt

A **.env** tartalmazza az Azure AI Agent Service kapcsolatláncát, az AOAI által használt modellt, valamint a megfelelő Google API Search szolgáltatás API-ját, ENDPOINT-ját stb.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Az Azure AI Agent Service Model Deployment neve"

[**NOTE**] Szükséged lesz egy modellre, amelynek 100,000-es Rate Limitje van (Tokenek percenként) és 600-as Rate Limitje (Kérések percenként).

  Modellt az Azure AI Foundry - Model and Endpoint részben szerezhetsz be.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Az Azure AI Agent Service Project Connection Stringje"

  A projekt kapcsolatláncát az AI ​​Foundry Portal képernyőjén, a projekt áttekintésében találhatod meg.

- **SERPAPI_SEARCH_API_KEY** = "A SERPAPI Search API KULCSOD"
- **SERPAPI_SEARCH_ENDPOINT** = "A SERPAPI Search Endpoint-ed"

Az Azure AI Agent Service Model Deployment Name és Project Connection String megszerzéséhez létre kell hoznod az Azure AI Agent Service-t. Ajánlott [ezt a sablont](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) használni a közvetlen létrehozáshoz (***Megjegyzés:*** Az Azure AI Agent Service jelenleg korlátozott régióban érhető el. Ajánlott, hogy [ezt a linket](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) tekintsd meg a régió beállításához).

Az Agent-nek hozzáférésre van szüksége a SERPAPI-hoz. Ajánlott [ezen a linken](https://serpapi.com/searches) regisztrálni. A regisztráció után egyedi API KULCSOT és ENDPOINT-ot szerezhetsz.


# Bejelentkezés az Azure-ba

Most be kell jelentkezned az Azure-ba. Nyiss meg egy terminált a VScode-ban, és futtasd az `az login` parancsot.


# Beállítás

Ennek a notebooknak a futtatásához telepítened kell a következő könyvtárakat. Itt található a szükséges könyvtárak listája és a hozzájuk tartozó pip telepítési parancsok:

azure-identity: Azure hitelesítéshez.  
requests: HTTP kérések küldéséhez.  
semantic-kernel: A szemantikai kernel keretrendszerhez (feltételezve, hogy ez egy egyedi vagy specifikus könyvtár, lehet, hogy egy adott forrásból vagy tárolóból kell telepíteni).  


In [ ]:
!pip install azure-identity
!pip install requests
!pip install semantic-kernel
!pip install --upgrade semantic_kernel
!pip install azure-cli

# Magyarázat:  
import asyncio: Ez importálja az asyncio modult, amely támogatja az aszinkron programozást Pythonban. Lehetővé teszi, hogy párhuzamos kódot írj az async és await szintaxis használatával.  
from typing import Annotated: Ez importálja az Annotated típust a typing modulból. Az Annotated arra szolgál, hogy metaadatokat adjunk a típusjelölésekhez, amelyek hasznosak lehetnek különböző célokra, például validációhoz, dokumentációhoz vagy eszközök számára.


In [ ]:
import asyncio,os
from typing import Annotated

# Magyarázat:
A from dotenv import load_dotenv és a load_dotenv() használatával könnyedén kezelheted a konfigurációs beállításokat és érzékeny információkat (például API kulcsokat és adatbázis URL-eket) egy .env fájlban, elkülönítve azokat a forráskódtól. Ezáltal az alkalmazásod biztonságosabbá és könnyebben konfigurálhatóvá válik.


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Magyarázat:

Importálási utasítás: from azure.identity.aio import DefaultAzureCredential: Ez importálja a DefaultAzureCredential osztályt az azure.identity.aio modulból. Az aio a modul nevében azt jelzi, hogy aszinkron műveletekre lett tervezve.

A DefaultAzureCredential célja: A DefaultAzureCredential osztály az Azure SDK for Python része. Ez egy alapértelmezett módot biztosít az Azure szolgáltatásokkal való hitelesítéshez. Több módszerrel próbál hitelesíteni egy meghatározott sorrendben, például környezeti változókkal, kezelt identitással és Azure CLI hitelesítő adatokkal.

Aszinkron műveletek: Az aio modul azt jelzi, hogy a DefaultAzureCredential osztály támogatja az aszinkron műveleteket. Ez azt jelenti, hogy használhatod az asyncio-val nem blokkoló hitelesítési kérések végrehajtására.


In [ ]:
from azure.identity.aio import DefaultAzureCredential

# Magyarázat:
Különböző modulokat és osztályokat importál a semantic_kernel csomagból. Íme az egyes importok részletes leírása:

AgentGroupChat a semantic_kernel.agents-ből: Ez az osztály az AI ügynökök csoportos csevegéséhez kapcsolódó funkciókat kezeli. AzureAIAgent és AzureAIAgentSettings a semantic_kernel.agents.azure_ai-ból:

AzureAIAgent: Ez az osztály Azure AI szolgáltatásokat használó AI ügynökök létrehozására és kezelésére szolgál.

AzureAIAgentSettings: Ez az osztály az AzureAIAgent beállításainak konfigurálására szolgál. TerminationStrategy a semantic_kernel.agents.strategies.termination.termination_strategy-ból:

Ez az osztály stratégiákat határoz meg az AI ügynökök végrehajtásának bizonyos feltételek melletti megszakítására. ChatMessageContent a semantic_kernel.contents.chat_message_content-ból:

Ez az osztály a csevegési üzenetek tartalmának kezelésére szolgál.
AuthorRole a semantic_kernel.contents.utils.author_role-ból:

Ez az osztály különböző szerepeket határoz meg a csevegési üzenetek szerzőinek kontextusában.

kernel_function a semantic_kernel.functions.kernel_function_decorator-ból: Ez a dekorátor kernel funkciók definiálására szolgál, amelyek a semantic kernel keretrendszeren belül végrehajthatók.
Ezek az importok előkészítik a szükséges komponenseket AI ügynökök létrehozásához és kezeléséhez, amelyek csoportos csevegési környezetben tudnak interakcióba lépni, például szállodafoglalás vagy hasonló tevékenységek céljából.


In [ ]:
from semantic_kernel.agents import AgentGroupChat
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent
from semantic_kernel.contents import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function

# Magyarázat:
Ezután importáljuk a CodeInterpreterTool osztályt az azure.ai.projects.models modulból.

CodeInterpreterTool: Ez az osztály az Azure AI SDK része, és arra szolgál, hogy kódot értelmezzen és hajtson végre AI projektek keretében. Funkciókat biztosít kódrészletek futtatására, kód elemzésére vagy a kód végrehajtásának integrálására AI munkafolyamatokban.  
Ez az importálás előkészíti a szükséges komponenst a CodeInterpreterTool használatához a projektben, amely hasznos lehet olyan feladatokhoz, amelyek dinamikusan értelmezik és hajtják végre a kódot.


In [ ]:
from azure.ai.projects.models import CodeInterpreterTool

# Magyarázat: 
Az ApprovalTerminationStrategy osztály egy speciális stratégiát biztosít egy AI ügynök működésének megszüntetésére. Az ügynök akkor áll le, ha az interakciós történetének utolsó üzenete tartalmazza a "mentve" szót. Ez hasznos lehet olyan helyzetekben, ahol az ügynök feladata befejezettnek tekinthető, amint megerősítést kap arról, hogy valami "mentve" lett. Határozza meg az interakciós módszert. Miután a foglalási terv mentésre került, leállítható, amikor megkapja a mentve jelet.


In [ ]:
class ApprovalTerminationStrategy(TerminationStrategy):
    """A strategy for determining when an agent should terminate."""

    async def should_agent_terminate(self, agent, history):
        """Check if the agent should terminate."""
        return "saved" in history[-1].content.lower()

# Magyarázat:

A kódsor egy AzureAIAgentSettings objektumot inicializál alapértelmezett vagy előre meghatározott beállításokkal a create() metódus meghívásával. Ez a beállításokat tartalmazó objektum (ai_agent_settings) ezután felhasználható egy AzureAIAgent példány konfigurálására és kezelésére.


In [ ]:
ai_agent_settings = AzureAIAgentSettings.create()

# Magyarázat:
A requests könyvtár importálásával egyszerűen végezhet HTTP kéréseket, és kapcsolatba léphet webes szolgáltatásokkal a Python kódjában.


In [ ]:
import requests

# Magyarázat:
Ez egy változó, amely az API-kulcsot tárolja egy SERP (Keresőmotor találati oldal) API szolgáltatás eléréséhez. Az API-kulcs egy egyedi azonosító, amelyet a fiókodhoz kapcsolódó kérések hitelesítésére használnak.

'GOOGLE_SEARCH_API_KEY': Ez egy helyőrző szöveg. Cseréld ki a ''GOOGLE_SEARCH_API_KEY'' értéket a tényleges SERP API-kulcsodra.

Cél: Ennek a sornak az a célja, hogy az API-kulcsot egy változóban tárolja, amelyet a SERP API szolgáltatás kéréseinek hitelesítésére lehet használni. Az API-kulcs szükséges a szolgáltatás eléréséhez és keresések végrehajtásához.

Hogyan szerezhetsz SERP API-kulcsot: Az API-kulcs megszerzéséhez kövesd az alábbi általános lépéseket a https://serpapi.com oldalon (a pontos lépések eltérhetnek az általad használt konkrét SERP API szolgáltatástól):

Válassz egy SERP API szolgáltatást: Számos SERP API szolgáltatás érhető el, például a SerpAPI, a Google Custom Search JSON API és mások. Válaszd ki azt, amelyik leginkább megfelel az igényeidnek.

Regisztrálj egy fiókot:

Látogass el a kiválasztott SERP API szolgáltatás weboldalára, például https://www.serpapi.com, és regisztrálj egy fiókot. Lehet, hogy meg kell adnod néhány alapvető információt, és meg kell erősítened az e-mail címedet.

Hozz létre egy API-kulcsot:

A regisztráció után jelentkezz be a fiókodba, és navigálj az API szekcióhoz vagy a vezérlőpulthoz. Keresd meg az új API-kulcs létrehozásának vagy generálásának lehetőségét.
Másold ki az API-kulcsot:

Miután az API-kulcsot létrehoztad, másold ki. Ezt a kulcsot fogod használni a SERP API szolgáltatás kéréseinek hitelesítésére.
Cseréld ki a helyőrzőt:

Cseréld ki a helyőrzőt a .env fájlodban.


In [ ]:
SERPAPI_SEARCH_API_KEY=os.getenv('SERPAPI_SEARCH_API_KEY')

In [ ]:
SERPAPI_SEARCH_ENDPOINT = os.getenv('SERPAPI_SEARCH_ENDPOINT')

# Magyarázat:
A BookingPlugin osztály olyan módszereket biztosít, amelyek segítségével szállodákat és repülőjáratokat lehet foglalni a Serpapi.com Google Search API használatával. Létrehozza a szükséges paramétereket, API-kéréseket küld, és feldolgozza a válaszokat, hogy releváns foglalási információkat adjon vissza. Az API-kulcs (SERPAPI_SEARCH_API_KEY) és az végpont (SERPAPI_SEARCH_ENDPOINT) azonosításra és a Google Search API-hoz való kérések küldésére szolgál.


In [ ]:
# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""
    @kernel_function(description="booking hotel")
    def booking_hotel(self,query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-in Time"])-> Annotated[str, "Return the result of booking hotel infomation"]:

        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "2",
            "currency": "USD",
            "gl": "us",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY
        }

        response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=params)
        if response.status_code == 200:
            response = response.json()
            return response["properties"]
        else:
            return None

    
    @kernel_function(description="booking fight")
    def  booking_fight(self,origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"])-> Annotated[str, "Return the result of booking fight infomation"]:
        
        go_params = {
            "engine": "google_flights",   
            "departure_id": origin,
            "arrival_id": destination,
            "outbound_date": outbound_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }

        print(go_params)

        go_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=go_params)


        result = ''

        if go_response.status_code == 200:
            response = go_response.json()

            result += "# outbound \n " + str(response)
        else:
            print('error!!!')
            # return None

        
        back_params = {
            "engine": "google_flights",   
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": return_date,
            "return_date": return_date,  
            "currency": "USD",
            "hl": "en",
            "api_key": SERPAPI_SEARCH_API_KEY  
        }


        print(back_params)


        back_response = requests.get(SERPAPI_SEARCH_ENDPOINT, params=back_params)



        if back_response.status_code == 200:
            response = back_response.json()

            result += "\n # return \n"  + str(response)

        else:
            print('error!!!')
            # return None
        
        print(result)

        return result

        


# Magyarázat:
A SavePlugin osztály egy saving_plan nevű metódust biztosít, amely lehetővé teszi az utazási tervek mentését az Azure AI szolgáltatások segítségével. Beállítja az Azure hitelesítő adatokat, létrehoz egy AI ügynököt, feldolgozza a felhasználói bemeneteket az utazási terv tartalmának létrehozásához és mentéséhez, valamint kezeli a fájlmentési és tisztítási műveleteket. A metódus sikeres befejezés esetén "Saved" értéket ad vissza.


In [ ]:
class SavePlugin:
    """Save Plugin for customers"""
    @kernel_function(description="saving plan")
    async def saving_plan(self,tripplan: Annotated[str, "The content of trip plan"])-> Annotated[str, "Return status of save content"]:

        async with (
            DefaultAzureCredential() as creds,
            AzureAIAgent.create_client(
                credential=creds,
                conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
            ) as client,
        ):

            code_interpreter = CodeInterpreterTool()
            
            agent_definition = await client.agents.create_agent(
                model=ai_agent_settings.model_deployment_name,
                tools=code_interpreter.definitions,
                tool_resources=code_interpreter.resources,
            )


            agent = AzureAIAgent(
                client=client,
                definition=agent_definition,
            )

            thread = await client.agents.create_thread()


            user_inputs = [
                """
            
                        You are my Python programming assistant. Generate code,save """+ tripplan +
                        
                    """    
                        and execute it according to the following requirements

                        1. Save blog content to trip-{YYMMDDHHMMSS}.md

                        2. give me the download this file link
                    """
            ]



            try:
                for user_input in user_inputs:
                    # Add the user input as a chat message
                    await agent.add_chat_message(
                        thread_id=thread.id, message=ChatMessageContent(role=AuthorRole.USER, content=user_input)
                    )
                    print(f"# User: '{user_input}'")
                    # Invoke the agent for the specified thread
                    async for content in agent.invoke(thread_id=thread.id):
                        if content.role != AuthorRole.TOOL:
                            print(f"# Agent: {content.content}")

                    
                    messages = await client.agents.list_messages(thread_id=thread.id)

                    # OpenAIPageableListOfThreadMessage
                    # OpenAIPageableListOfThreadMessage


                    for file_path_annotation in messages.file_path_annotations:

                            file_name = os.path.basename(file_path_annotation.text)

                            await client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./trip")

                    
            finally:
                await client.agents.delete_thread(thread.id)
                await client.agents.delete_agent(agent.id)


        return "Saved"

# Magyarázat:
Ez a kód Azure AI ügynököket állít be repülőjegyek és szállodák foglalására, valamint utazási tervek mentésére a felhasználói bemenetek alapján. Azure hitelesítő adatokat használ az ügynökök létrehozásához és konfigurálásához, a felhasználói bemeneteket egy csoportos csevegésen keresztül dolgozza fel, és biztosítja a megfelelő takarítást a feladatok befejezése után. Az ügynökök speciális bővítményeket (BookingPlugin és SavePlugin) használnak a saját feladataik elvégzéséhez.


In [ ]:
async with (
    DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):
    BOOKING_AGENT_NAME = "BookingAgent"
    BOOKING_AGENT_INSTRUCTIONS = """
    You are a booking agent. Help me book flights or hotels.

    Thought: Please understand the user's intention and confirm whether to use the reservation system to complete the task.

    Actions:
    - For flight bookings, convert the departure and destination names into airport codes.
    - Use the appropriate API for hotel or flight bookings. Verify that all necessary parameters are available. If any parameters are missing, ask the user to provide them. If all parameters are complete, call the corresponding function.
    - If the task is not related to hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
      - For flight bookings, output separate outbound and return contents in the order of:
        Departure Airport | Airline | Flight Number | Departure Time | Arrival Airport | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
      - For hotel bookings, output in the order of:
        Property Name | Property Description | Check-in Time | Check-out Time | Prices | Nearby Places | Hotel Class | GPS Coordinates.
    """

    SAVE_AGENT_NAME = "SaveAgent"
    SAVE_AGENT_INSTRUCTIONS = """
    You are a save tool agent. Help me to save the trip plan.
    """

    # Create agent definition
    booking_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=BOOKING_AGENT_NAME,
        instructions=BOOKING_AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent
    booking_agent = AzureAIAgent(
        client=client,
        definition=booking_agent_definition,
        # Optionally configure polling options
        # polling_options=RunPollingOptions(run_polling_interval=timedelta(seconds=1)),
    )

    # Add the sample plugin to the kernel
    booking_agent.kernel.add_plugin(BookingPlugin(), plugin_name="booking")

    # Create agent definition
    save_agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=SAVE_AGENT_NAME,
        instructions=SAVE_AGENT_INSTRUCTIONS
    )

    # Create the AzureAI Agent
    save_agent = AzureAIAgent(
        client=client,
        definition=save_agent_definition,
    )

    save_agent.kernel.add_plugin(SavePlugin(), plugin_name="saving")

    user_inputs = [
        "I have a business trip from London to New York in Feb 20 2025 to Feb 27 2025 ,help me to book a hotel and fight tickets and save it"
    ]

    chat = AgentGroupChat(
        agents=[booking_agent, save_agent],
        termination_strategy=ApprovalTerminationStrategy(agents=[save_agent], maximum_iterations=10),
    )

    try:
        for user_input in user_inputs:
            # Add the user input as a chat message
            await chat.add_chat_message(
                ChatMessageContent(role=AuthorRole.USER, content=user_input)
            )
            print(f"# User: '{user_input}'")

            async for content in chat.invoke():
                print(f"# {content.role} - {content.name or '*'}: '{content.content}'")

            print(f"# IS COMPLETE: {chat.is_complete}")

            print("*" * 60)
            print("Chat History (In Descending Order):\n")
            async for message in chat.get_chat_messages(agent=save_agent):
                print(f"# {message.role} - {message.name or '*'}: '{message.content}'")
    finally:
        await chat.reset()
        await client.agents.delete_agent(save_agent.id)
        await client.agents.delete_agent(booking_agent.id)



---

**Felelősség kizárása**:  
Ez a dokumentum az AI fordítási szolgáltatás, a [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével lett lefordítva. Bár törekszünk a pontosságra, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum az eredeti nyelvén tekintendő hiteles forrásnak. Kritikus információk esetén javasolt professzionális emberi fordítást igénybe venni. Nem vállalunk felelősséget semmilyen félreértésért vagy téves értelmezésért, amely a fordítás használatából eredhet.
